In [55]:
import pandas as pd
import re
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error


In [56]:
data = pd.read_csv("../data/ml_gw_car_insurance.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97655 entries, 0 to 97654
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        97655 non-null  int64  
 1   policy_id                         97655 non-null  object 
 2   policy_tenure                     97655 non-null  float64
 3   age_of_car                        97655 non-null  float64
 4   age_of_policyholder               97655 non-null  float64
 5   area_cluster                      97655 non-null  object 
 6   population_density                97655 non-null  int64  
 7   make                              97655 non-null  int64  
 8   segment                           97655 non-null  object 
 9   model                             97655 non-null  object 
 10  fuel_type                         97655 non-null  object 
 11  max_torque                        97655 non-null  object 
 12  max_

## Max Torque and Max Power (Only Numbers)


In [57]:
data["max_torque"].unique()

array(['60Nm@3500rpm', '113Nm@4400rpm', '91Nm@4250rpm', '250Nm@2750rpm',
       '200Nm@3000rpm', '82.1Nm@3400rpm', '200Nm@1750rpm', '85Nm@3000rpm',
       '170Nm@4000rpm'], dtype=object)

In [58]:
data['torque_Nm'] = data['max_torque'].str.extract('(\d+)').astype(float)
data['torque_rpm'] = data['max_torque'].str.extract('@(\d+)').astype(float)

data[["max_torque", "torque_Nm", "torque_rpm"]]

,max_torque,torque_Nm,torque_rpm
0,60Nm@3500rpm,60.0,3500.0
1,60Nm@3500rpm,60.0,3500.0
2,60Nm@3500rpm,60.0,3500.0
3,113Nm@4400rpm,113.0,4400.0
4,91Nm@4250rpm,91.0,4250.0
...,...,...,...
97650,113Nm@4400rpm,113.0,4400.0
97651,250Nm@2750rpm,250.0,2750.0
97652,113Nm@4400rpm,113.0,4400.0
97653,113Nm@4400rpm,113.0,4400.0


In [59]:
data["max_power"].unique()

array(['40.36bhp@6000rpm', '88.50bhp@6000rpm', '67.06bhp@5500rpm',
       '113.45bhp@4000rpm', '88.77bhp@4000rpm', '55.92bhp@5300rpm',
       '97.89bhp@3600rpm', '61.68bhp@6000rpm', '118.36bhp@5500rpm'],
      dtype=object)

In [60]:
data['power_bhp'] = data['max_power'].str.extract('(\d+.\d+)').astype(float)
data['power_rpm'] = data['max_power'].str.extract('@(\d+)').astype(float)

data[["max_power", "power_bhp", "power_rpm"]]

,max_power,power_bhp,power_rpm
0,40.36bhp@6000rpm,40.36,6000.0
1,40.36bhp@6000rpm,40.36,6000.0
2,40.36bhp@6000rpm,40.36,6000.0
3,88.50bhp@6000rpm,88.50,6000.0
4,67.06bhp@5500rpm,67.06,5500.0
...,...,...,...
97650,88.50bhp@6000rpm,88.50,6000.0
97651,113.45bhp@4000rpm,113.45,4000.0
97652,88.50bhp@6000rpm,88.50,6000.0
97653,88.50bhp@6000rpm,88.50,6000.0


## Max Torque and Max Power (Dummies)


In [61]:
#Max Torque Dummies
torque_dummies = pd.get_dummies(data['max_torque'], prefix='torque')
data = pd.concat([data, torque_dummies], axis=1)

data.head()

,Unnamed: 0,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,...,power_rpm,torque_113Nm@4400rpm,torque_170Nm@4000rpm,torque_200Nm@1750rpm,torque_200Nm@3000rpm,torque_250Nm@2750rpm,torque_60Nm@3500rpm,torque_82.1Nm@3400rpm,torque_85Nm@3000rpm,torque_91Nm@4250rpm
0,0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,...,6000.0,0,0,0,0,0,1,0,0,0
1,1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,...,6000.0,0,0,0,0,0,1,0,0,0
2,2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,...,6000.0,0,0,0,0,0,1,0,0,0
3,3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,...,6000.0,1,0,0,0,0,0,0,0,0
4,4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,...,5500.0,0,0,0,0,0,0,0,0,1


In [62]:
#Max Power Dummies
power_dummies = pd.get_dummies(data['max_power'], prefix='power')
data = pd.concat([data, power_dummies], axis=1)

data.head()

,Unnamed: 0,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,...,torque_91Nm@4250rpm,power_113.45bhp@4000rpm,power_118.36bhp@5500rpm,power_40.36bhp@6000rpm,power_55.92bhp@5300rpm,power_61.68bhp@6000rpm,power_67.06bhp@5500rpm,power_88.50bhp@6000rpm,power_88.77bhp@4000rpm,power_97.89bhp@3600rpm
0,0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,...,0,0,0,1,0,0,0,0,0,0
1,1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,...,0,0,0,1,0,0,0,0,0,0
2,2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,...,0,0,0,1,0,0,0,0,0,0
3,3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,...,0,0,0,0,0,0,0,1,0,0
4,4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,...,1,0,0,0,0,0,1,0,0,0


## Binary varianbles for Yes and No Variables

In [63]:
columns = ['is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors', 'is_parking_camera','is_front_fog_lights','is_rear_window_wiper','is_rear_window_washer','is_rear_window_defogger','is_brake_assist','is_power_door_locks','is_central_locking','is_power_steering','is_driver_seat_height_adjustable','is_day_night_rear_view_mirror','is_ecw','is_speed_alert']
for col in columns:
    data[col] = data[col].map({'Yes': 1, 'No': 0})

data[['is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors', 'is_parking_camera','is_front_fog_lights','is_rear_window_wiper','is_rear_window_washer','is_rear_window_defogger','is_brake_assist','is_power_door_locks','is_central_locking','is_power_steering','is_driver_seat_height_adjustable','is_day_night_rear_view_mirror','is_ecw','is_speed_alert']]

,is_esc,is_adjustable_steering,is_tpms,is_parking_sensors,is_parking_camera,is_front_fog_lights,is_rear_window_wiper,is_rear_window_washer,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert
0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
3,1,1,0,1,1,1,0,0,1,1,1,1,1,1,1,1,1
4,0,0,0,0,1,0,0,0,0,0,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97650,0,1,0,1,0,1,0,0,0,1,1,1,1,1,1,1,1
97651,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1
97652,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
97653,1,1,0,1,1,1,0,0,1,1,1,1,1,1,1,1,1


## Dummies for Fuel Type, Rear Brakes Type and Transmission Type

In [64]:
#Fuel Type
fuel_type_dummies = pd.get_dummies(data['fuel_type'], prefix='fuel_type')
data = pd.concat([data, fuel_type_dummies], axis=1)


#Rear Brakes
rear_brakes_dummies = pd.get_dummies(data['rear_brakes_type'], prefix='rear_brakes')
data = pd.concat([data, rear_brakes_dummies], axis=1)


#Transmission Type
transmission_type_dummies = pd.get_dummies(data['transmission_type'], prefix='transmission_type')
data = pd.concat([data, transmission_type_dummies], axis=1)

#Segment
segment_dummies = pd.get_dummies(data['segment'], prefix='segment')
data = pd.concat([data, segment_dummies], axis=1)

# steering_type
steering_type_dummies = pd.get_dummies(data['steering_type'], prefix='steering_type')
data = pd.concat([data, steering_type_dummies], axis=1)

data.head()


,Unnamed: 0,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,...,transmission_type_Manual,segment_A,segment_B1,segment_B2,segment_C1,segment_C2,segment_Utility,steering_type_Electric,steering_type_Manual,steering_type_Power
0,0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,...,1,1,0,0,0,0,0,0,0,1
1,1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,...,1,1,0,0,0,0,0,0,0,1
2,2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,...,1,1,0,0,0,0,0,0,0,1
3,3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,...,0,0,0,0,1,0,0,1,0,0
4,4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,...,0,1,0,0,0,0,0,1,0,0


In [65]:
# Define regular expression pattern to match only digits
pattern = r'\d+'

# Apply regular expression to 'cluster' column and save as new column
data['area_cluster'] = data['area_cluster'].apply(lambda x: re.search(pattern, x).group())
data['model'] = data['model'].apply(lambda x: re.search(pattern, x).group())

data.head()

,Unnamed: 0,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,...,transmission_type_Manual,segment_A,segment_B1,segment_B2,segment_C1,segment_C2,segment_Utility,steering_type_Electric,steering_type_Manual,steering_type_Power
0,0,ID00001,0.515874,0.05,0.644231,1,4990,1,A,1,...,1,1,0,0,0,0,0,0,0,1
1,1,ID00002,0.672619,0.02,0.375000,2,27003,1,A,1,...,1,1,0,0,0,0,0,0,0,1
2,2,ID00003,0.841110,0.02,0.384615,3,4076,1,A,1,...,1,1,0,0,0,0,0,0,0,1
3,3,ID00004,0.900277,0.11,0.432692,4,21622,1,C1,2,...,0,0,0,0,1,0,0,1,0,0
4,4,ID00005,0.596403,0.11,0.634615,5,34738,2,A,3,...,0,1,0,0,0,0,0,1,0,0


In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97655 entries, 0 to 97654
Data columns (total 83 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        97655 non-null  int64  
 1   policy_id                         97655 non-null  object 
 2   policy_tenure                     97655 non-null  float64
 3   age_of_car                        97655 non-null  float64
 4   age_of_policyholder               97655 non-null  float64
 5   area_cluster                      97655 non-null  object 
 6   population_density                97655 non-null  int64  
 7   make                              97655 non-null  int64  
 8   segment                           97655 non-null  object 
 9   model                             97655 non-null  object 
 10  fuel_type                         97655 non-null  object 
 11  max_torque                        97655 non-null  object 
 12  max_

In [67]:
data.dropna(inplace=True)
# data.drop(columns=['index', 'Unnamed: 0'], axis = 1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58592 entries, 0 to 58591
Data columns (total 83 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        58592 non-null  int64  
 1   policy_id                         58592 non-null  object 
 2   policy_tenure                     58592 non-null  float64
 3   age_of_car                        58592 non-null  float64
 4   age_of_policyholder               58592 non-null  float64
 5   area_cluster                      58592 non-null  object 
 6   population_density                58592 non-null  int64  
 7   make                              58592 non-null  int64  
 8   segment                           58592 non-null  object 
 9   model                             58592 non-null  object 
 10  fuel_type                         58592 non-null  object 
 11  max_torque                        58592 non-null  object 
 12  max_

In [68]:
# Create a list of all possible variables 
list_elements = data.columns.values.tolist()
list_elements.remove('is_claim')
list_elements.remove('segment')
list_elements.remove('model')
list_elements.remove('fuel_type')
list_elements.remove('max_torque')
list_elements.remove('max_power')
list_elements.remove('engine_type')
list_elements.remove('rear_brakes_type')
list_elements.remove('steering_type')
list_elements.remove('policy_id')
list_elements.remove('Unnamed: 0')
list_elements.remove('transmission_type')
list_elements

['policy_tenure',
 'age_of_car',
 'age_of_policyholder',
 'area_cluster',
 'population_density',
 'make',
 'airbags',
 'is_esc',
 'is_adjustable_steering',
 'is_tpms',
 'is_parking_sensors',
 'is_parking_camera',
 'displacement',
 'cylinder',
 'gear_box',
 'turning_radius',
 'length',
 'width',
 'height',
 'gross_weight',
 'is_front_fog_lights',
 'is_rear_window_wiper',
 'is_rear_window_washer',
 'is_rear_window_defogger',
 'is_brake_assist',
 'is_power_door_locks',
 'is_central_locking',
 'is_power_steering',
 'is_driver_seat_height_adjustable',
 'is_day_night_rear_view_mirror',
 'is_ecw',
 'is_speed_alert',
 'ncap_rating',
 'torque_Nm',
 'torque_rpm',
 'power_bhp',
 'power_rpm',
 'torque_113Nm@4400rpm',
 'torque_170Nm@4000rpm',
 'torque_200Nm@1750rpm',
 'torque_200Nm@3000rpm',
 'torque_250Nm@2750rpm',
 'torque_60Nm@3500rpm',
 'torque_82.1Nm@3400rpm',
 'torque_85Nm@3000rpm',
 'torque_91Nm@4250rpm',
 'power_113.45bhp@4000rpm',
 'power_118.36bhp@5500rpm',
 'power_40.36bhp@6000rpm',
 'po

In [78]:
import warnings
from sklearn.exceptions import DataConversionWarning


def best_linear():

    # Ignore the DataConversionWarning
    warnings.filterwarnings("ignore", category=DataConversionWarning)

    kept_2 = 10
    kept_i_2 = 0
    max = 0

    list_elements2 = list_elements.copy()
    list_of_elements = []

    while True:
        # Check which is the next best element to add 
        for i in list_elements2:
            temp = list_of_elements.copy()
            temp.append(i)
            
            def regression_f(input, price):
                model = linear_model.LinearRegression()
                return -1*cross_val_score(model, input, price, scoring='neg_mean_squared_error', cv=10).mean()
            test_error = regression_f(data[temp], data[['is_claim']])
            print(test_error)
            if test_error < kept_2:
                kept_2 = test_error
                kept_i_2 = i 

        # stop adding elements when the error is not better than previosly obtained
        if kept_i_2 not in list_elements2:
            break

        list_of_elements.append(kept_i_2)
        list_elements2.remove(kept_i_2)

    return kept_2, list_of_elements

In [79]:
m,n = best_linear()
print(f'Linear model has error {m}, with corresponding variables {n}')

0.05950978707708996
0.059833863110646826
0.05984958202639825
0.05988111442943082
0.059860537733451714
0.05988007261619631
0.05987899541653301
0.059878663799571984
0.05986885567298048
0.059879183331641606
0.059876415523629524
0.05987933250596533
0.05987617336673362
0.05986950471523696
0.0598792845908962
0.05987911729137465
0.05987705584231625
0.05987406397157989
0.059879246080468594
0.0598783543816012
0.059872386271521486
0.05987897482780362
0.05987897482780362
0.05987889414886214
0.059873103774094785
0.05987751506052571
0.05987751506052571
0.05987836844629134
0.059874030369893784
0.05987772641668694
0.05987751506052571
0.05987540977186486
0.05987835052347747
0.05987844639304145
0.059878320192124336
0.05987646621034394
0.05987943168951435
0.05987112810637187
0.05987540977186486
0.0598799768695469
0.05987978673567572
0.059879183331641606
0.059877846437896044
0.059879563831217696
0.05987836844629134
0.059876415523629524
0.059879183331641606
0.05987540977186486
0.059877846437896044
0.05987